In [1]:
from ltr.client import OpenSearchClient
client = OpenSearchClient()

http://localhost:9201/_ltr; <OpenSearch([{'host': 'localhost', 'port': 9201}])>


In [2]:
from ltr import download
corpus='http://es-learn-to-rank.labs.o19s.com/tmdb.json'
judgments='http://es-learn-to-rank.labs.o19s.com/title_judgments.txt'

download([corpus, judgments], dest='data/');

data/tmdb.json already exists
data/title_judgments.txt already exists


In [4]:
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

movies=indexable_movies(movies='data/tmdb.json')
rebuild(client, index='tmdb', doc_src=movies)

Index tmdb already exists. Use `force = True` to delete and recreate


In [5]:
client.reset_ltr(index='tmdb')

config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }
    
           },
           "featureset": {
            "features": [
                { #1
                    "name": "title_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"title": "{{keywords}}"}
                    }
                },
                { #2
                    "name": "overview_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"overview": "{{keywords}}"}
                    }
                }
            ]
    }}


client.create_featureset(index='tmdb', name='movies', ftr_config=config)

Removed Default LTR feature store [Status: 200]
Initialize Default LTR feature store [Status: 200]
Create movies feature set [Status: 201]


In [6]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movies')
with judgments_open('data/title_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))

Recognizing 40 queries


In [7]:
from ltr.ranklib import train
trainLog  = train(client,
                  training_set=ftr_logger.logged,
                  index='tmdb',
                  featureSet='movies',
                  modelName='title')

/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t DCG@10 -tree 50 -bag 1 -leaf 10 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/title_model.txt 
Delete model title: 404
Created Model title [Status: 201]
Model saved


In [8]:
from ltr import search
search(client, "cartoon with basketball aliens", modelName='title')

{"size": 5, "query": {"sltr": {"params": {"keywords": "cartoon with basketball aliens", "keywordsList": ["cartoon with basketball aliens"]}, "model": "title"}}}
{'size': 5, 'query': {'sltr': {'params': {'keywords': 'cartoon with basketball aliens', 'keywordsList': ['cartoon with basketball aliens']}, 'model': 'title'}}}
Love & Basketball 
2.798212 
2000 
['Action', 'Comedy', 'Drama', 'Romance'] 
A young African-American couple navigates the tricky paths of romance and athletics in this drama. Quincy McCall (Omar Epps) and Monica Wright (Sanaa Lathan) grew up in the same neighborhood and have known each other since childhood. As they grow into adulthood, they fall in love, but they also share another all-consuming passion: basketball. They've followed the game all their lives and have no small amount of talent on the court. As Quincy and Monica struggle to make their relationship work, they follow separate career paths though high school and college basketball and, they hope, into stard